# to test if y models make sense, we need to test it on a dsQTL benchmark
first get the same subset as delta SVM

In [3]:
import pandas as pd
qtls = pd.read_csv('/data1/lesliec/sarthak/data/joint_playground/dsQTL/GSE31388_dsQtlTableLong.txt', sep='\t')
qtls

,Chr,Start,End,SNP,Estimate,Std. Error,t value,Pr(>|t|)
0,chr1,795000,795100.0,783384,1.8810,0.45920,4.097,1.138000e-04
1,chr1,846400,846500.0,834961,0.4790,0.09055,5.290,1.406000e-06
2,chr1,865600,865700.0,859335,0.4703,0.11320,4.155,9.291000e-05
3,chr1,877200,877300.0,889920,-1.3300,0.26390,-5.040,3.676000e-06
4,chr1,878100,878200.0,880884,-0.6849,0.15050,-4.552,2.261000e-05
...,...,...,...,...,...,...,...,...
8897,chr22,49391500,49391600.0,49383656,0.7830,0.15800,4.955,5.068000e-06
8898,chr22,49391600,49391700.0,49390483,1.4380,0.19260,7.468,2.021000e-10
8899,chr22,49398100,49398200.0,49404768,-0.6065,0.13590,-4.463,3.123000e-05
8900,chr22,49405900,49406000.0,49386579,-0.6270,0.14730,-4.257,6.498000e-05


In [4]:
#first subset to SNPs between start and end
keep = []
for i in range(len(qtls)):
    if qtls['Start'][i] < qtls['SNP'] and qtls['End'][i] > qtls['SNP']:
        keep.append(i)
len(keep)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [5]:
#this might be the wrong genome lmao, let's see what it is
import numpy as np
genome = np.load('/data1/lesliec/sarthak/data/chrombpnet_test/hg38_tokenized.npz')

genome['chr1'][846446]

9

In [6]:
genome['chr1'][856583]

9

In [7]:
#yeah so it should be A... it's because their version is HG18
#we can use liftover! Wait bim files are better

bim_path = '/data1/deyk/extras/1000G_BIMS_hg19/1000G.EUR.QC.1.bim'
bim = pd.read_csv(bim_path, sep='\t', header=None)

#let's go through some example rsids
rs1 = 'rs7417106'
bim

,0,1,2,3,4,5
0,1,rs575272151,0.000,11008,G,C
1,1,rs544419019,0.000,11012,G,C
2,1,rs540538026,0.000,13110,A,G
3,1,rs62635286,0.000,13116,G,T
4,1,rs200579949,0.000,13118,G,A
...,...,...,...,...,...,...
779349,1,rs374472180,293.397,249233164,C,T
779350,1,rs548489155,293.397,249236809,C,T
779351,1,rs569532168,293.397,249236810,A,T
779352,1,rs573070055,293.397,249239303,T,C


In [8]:
#now look for the rows in column 1 which contain rs1
rs1_rows = bim[bim[1].str.contains(rs1)]
rs1_rows

,0,1,2,3,4,5
1052,1,rs7417106,0.681682,911595,A,G


In [9]:
#now load in the bim path for hg38 too

bim_path = '/data1/deyk/extras/1000G_BIMS_hg38/1000G.EUR.QC.1.bim'

bim = pd.read_csv(bim_path, sep='\t', header=None)
rs1_rows = bim[bim[1].str.contains(rs1)]
rs1_rows

,0,1,2,3,4,5
1052,1,rs7417106,0.681682,976215,A,G


In [10]:
#it's different but the same rsid, so we can check the genome
genome = np.load('/data1/lesliec/sarthak/data/chrombpnet_test/hg38_tokenized.npz')
genome['chr1'][976215] #uh it's a C?? it should theoretically be A or G...

8

In [11]:
#reference should be A we found by looking it up!
#let's look at another example
rs2 = 'rs6603785'
bim[bim[1].str.contains(rs2)]

,0,1,2,3,4,5
2500,1,rs6603785,1.190243,1251122,T,A


In [12]:
#and if we check the genome
genome['chr1'][1251122] #this is again a C, when it should be a Ta or A...

#wait a minute, is this an issue of indexing via 1 or 0?


8

In [13]:
genome['chr1'][1251122-1] #ahh so it's A? Yes indeed, reference is A, but bim shows T?

7

In [14]:
#let's check the excel file and make sure it matches
vars_list = pd.read_excel('/data1/lesliec/sarthak/data/joint_playground/dsQTL/41588_2015_BFng3331_MOESM26_ESM.xlsx', sheet_name='SuppTable1')
vars_list

,chrom_hg18,pos_hg18,SNPname1,chrom_hg19,pos_hg19,SNPname2,allele1,allele2,gkm_SVM,kmer_SVM,abs_gkm_SVM,abs_kmer_SVM,CADD,GWAVA_region,GWAVA_tss,GWAVA_unmatched,GERP,label
0,chr1,846446,chr1.846446,chr1,856583,rs141671872,A,G,2.653531,7.938784,2.653531,7.938784,-1.522533,0.48,0.53,0.75,-0.682,1
1,chr1,901458,chr1.901458,chr1,911595,rs7417106,G,A,2.821422,6.027648,2.821422,6.027648,1.739152,0.29,0.36,0.89,2.150,1
2,chr1,1176365,chr1.1176365,chr1,1186502,rs6603785,T,A,-7.167236,-10.870018,7.167236,10.870018,-0.091378,0.42,0.49,0.66,0.451,1
3,chr1,1217275,chr1.1217275,chr1,1227412,rs112571439,A,G,-3.957382,2.387491,3.957382,2.387491,1.127472,0.34,0.43,0.49,2.430,1
4,chr1,1580438,chr1.1580438,chr1,1590575,rs9661285,A,G,-4.785596,-6.827102,4.785596,6.827102,0.665464,0.38,0.35,0.86,1.640,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28304,chr9,140155407,chr9.140155407,chr9,141035586,rs7852209,T,C,0.444036,-1.583811,0.444036,1.583811,-0.606667,0.19,0.22,0.25,-0.890,-1
28305,chr9,140181121,chr9.140181121,chr9,141061300,rs4066694,T,C,1.363332,1.703144,1.363332,1.703144,0.489617,0.42,0.14,0.16,0.118,-1
28306,chr9,140213801,chr9.140213801,chr9,141093980,rs7046723,A,G,-3.102628,3.391616,3.102628,3.391616,-0.744948,0.23,0.18,0.35,0.000,-1
28307,chr9,140219484,chr9.140219484,chr9,141099663,rs12340415,G,C,-4.628759,-2.087131,4.628759,2.087131,-0.809512,0.41,0.27,0.15,0.000,-1


In [40]:
#let's load in the bim files
bims = {}
for i in range(1,23):
    bim_path = f'/data1/deyk/extras/1000G_BIMS_hg38/1000G.EUR.QC.{i}.bim'
    bim = pd.read_csv(bim_path, sep='\t', header=None)
    bims[i] = bim
bims.keys

<function dict.keys>

In [41]:
from tqdm import tqdm
no_match = []
for i in tqdm(range(vars_list.shape[0])):
    rsid = vars_list['SNPname2'][i]
    chrom = int(vars_list['chrom_hg19'][i][3:])
    # print(rsid,chrom)
    # break
    subset = bims[chrom][bims[chrom][1].str.contains(rsid)]
    if subset.shape[0] == 0:
        no_match.append(rsid)
        continue
len(no_match) # 0, so all match

  0%|          | 15/28309 [00:02<1:15:28,  6.25it/s]


KeyboardInterrupt: 

In [42]:
# Step 1: Build rsID sets from each bim
bim_rsids = {chrom: set(bims[chrom][1].values) for chrom in bims}

# Step 2: Check membership
def is_rsid_in_bim(row):
    chrom = int(row['chrom_hg19'][3:])
    rsid = row['SNPname2']
    return rsid in bim_rsids.get(chrom, set())

# Step 3: Apply the function to the dataframe
vars_list['match'] = vars_list.apply(is_rsid_in_bim, axis=1)

# Step 4: Extract rsIDs with no match
no_match = vars_list.loc[~vars_list['match'], 'SNPname2'].tolist()


In [43]:
len(no_match) #why do half of them not match??

10794

In [ ]:
vars_list['match'] #yeah just manual checking we saw simialr things...

0        False
1         True
2         True
3        False
4        False
         ...  
28304     True
28305     True
28306    False
28307     True
28308    False
Name: match, Length: 28309, dtype: bool

In [ ]:
#now make sure chromosomes match!

for i in tqdm(range(vars_list.shape[0])):
    if i < 10:
        continue
    if not vars_list['match'][i]:
        continue
    #now you get the snp, make sure it matches the bim
    rsid = vars_list['SNPname2'][i]
    chrom = int(vars_list['chrom_hg19'][i][3:])
    # print(rsid,chrom)
    # break
    subset = bims[chrom][bims[chrom][1].str.contains(rsid)]
    if subset.shape[0] == 0:
        continue
    print(subset)
    print(vars_list.iloc[i])
    print(genome[f'chr{chrom}'][subset[3].values[0]-1]) #this is the genome position
    break
#so it seems like it mostly matches allele 2, let's ensure it alway does

  0%|          | 0/28309 [00:00<?, ?it/s]

       0           1          2         3  4  5
95664  1  rs79931859  49.032714  25549849  A  G
chrom_hg18                  chr1
pos_hg18                25748927
SNPname1           chr1.25748927
chrom_hg19                  chr1
pos_hg19                25876340
SNPname2              rs79931859
allele1                        A
allele2                        G
gkm_SVM                -1.480844
kmer_SVM               -6.299706
abs_gkm_SVM             1.480844
abs_kmer_SVM            6.299706
CADD                    0.115655
GWAVA_region                0.44
GWAVA_tss                   0.47
GWAVA_unmatched             0.84
GERP                       -1.22
label                          1
match                       True
Name: 11, dtype: object


  0%|          | 11/28309 [00:00<16:24, 28.76it/s]

9


In [49]:
mapping = {7:'A', 8:'C', 9:'G', 10:'T'}
for i in tqdm(range(vars_list.shape[0])):
    if not vars_list['match'][i]:
        continue
    #now you get the snp, make sure it matches the bim
    rsid = vars_list['SNPname2'][i]
    chrom = int(vars_list['chrom_hg19'][i][3:])
    # print(rsid,chrom)
    # break
    subset = bims[chrom][bims[chrom][1].str.contains(rsid)]
    if subset.shape[0] == 0:
        continue
    # print(subset)
    # print(vars_list.iloc[i])
    # print(genome[f'chr{chrom}'][subset[3].values[0]-1]) #this is the genome position
    # break
    nucleotide = genome[f'chr{chrom}'][subset[3].values[0]-1]
    if vars_list['allele2'][i] != mapping[nucleotide]:
        print(i)
        break

  0%|          | 0/28309 [00:00<?, ?it/s]

  0%|          | 7/28309 [00:01<2:07:30,  3.70it/s]

7


In [50]:
vars_list['allele2'][i]

'G'

In [51]:
mapping[nucleotide]

'A'

In [ ]:
print(subset)
print(vars_list.iloc[i])
print(genome[f'chr{chrom}'][subset[3].values[0]-1])

#so it seems like it's one of them, but which one it is is random?

       0          1          2         3  4  5
45548  1  rs2236055  24.228448  11982204  G  A
chrom_hg18                  chr1
pos_hg18                11964848
SNPname1           chr1.11964848
chrom_hg19                  chr1
pos_hg19                12042261
SNPname2               rs2236055
allele1                        A
allele2                        G
gkm_SVM                 2.366937
kmer_SVM                1.784808
abs_gkm_SVM             2.366937
abs_kmer_SVM            1.784808
CADD                    3.482362
GWAVA_region                0.23
GWAVA_tss                   0.22
GWAVA_unmatched             0.64
GERP                        2.11
label                          1
match                       True
Name: 7, dtype: object
7


In [56]:
mapping = {7:'A', 8:'C', 9:'G', 10:'T'}
for i in tqdm(range(vars_list.shape[0])):
    if not vars_list['match'][i]:
        continue
    rsid = vars_list['SNPname2'][i]
    chrom = int(vars_list['chrom_hg19'][i][3:])
    subset = bims[chrom][bims[chrom][1].str.contains(rsid)]
    if subset.shape[0] == 0:
        continue
    nucleotide = genome[f'chr{chrom}'][subset[3].values[0]-1]
    if vars_list['allele2'][i] != mapping[nucleotide] and vars_list['allele1'][i] != mapping[nucleotide]:
        print(i)
        break

  0%|          | 0/28309 [00:00<?, ?it/s]

  0%|          | 137/28309 [00:26<1:31:37,  5.12it/s]

137


In [ ]:
#why tf would it stop?

print(subset)
print(vars_list.iloc[i])
print(genome[f'chr{chrom}'][subset[3].values[0]-1])

         0          1          2          3  4  5
364339  13  rs7338868  129.52588  113931211  G  T
chrom_hg18             chr13
pos_hg18           113479759
SNPname1           rs7338868
chrom_hg19             chr13
pos_hg19           114634184
SNPname2           rs7338868
allele1                    A
allele2                    C
gkm_SVM            17.609279
kmer_SVM           12.627238
abs_gkm_SVM        17.609279
abs_kmer_SVM       12.627238
CADD               -0.132208
GWAVA_region            0.34
GWAVA_tss               0.38
GWAVA_unmatched         0.23
GERP                   -2.29
label                      1
match                   True
Name: 137, dtype: object
10


In [58]:
subset[4].values[0] #this is the allele 1

'G'

In [59]:
#so in this case we match the bim file, not this weird thing, unsure why this would not match, but let's keep track of it and keep going

not_matched = []
mapping = {7:'A', 8:'C', 9:'G', 10:'T'}
for i in tqdm(range(vars_list.shape[0])):
    if not vars_list['match'][i]:
        continue
    rsid = vars_list['SNPname2'][i]
    chrom = int(vars_list['chrom_hg19'][i][3:])
    subset = bims[chrom][bims[chrom][1].str.contains(rsid)]
    if subset.shape[0] == 0:
        continue
    nucleotide = genome[f'chr{chrom}'][subset[3].values[0]-1]
    if vars_list['allele2'][i] != mapping[nucleotide] and vars_list['allele1'][i] != mapping[nucleotide]:
        not_matched.append(i)
        if subset[4].values[0] != mapping[nucleotide] and vars_list['allele1'][i] != mapping[nucleotide]:
            print(i)
            print(subset)
            print(vars_list.iloc[i])
            print(genome[f'chr{chrom}'][subset[3].values[0]-1])
            break

  0%|          | 137/28309 [00:25<1:28:43,  5.29it/s]

137
         0          1          2          3  4  5
364339  13  rs7338868  129.52588  113931211  G  T
chrom_hg18             chr13
pos_hg18           113479759
SNPname1           rs7338868
chrom_hg19             chr13
pos_hg19           114634184
SNPname2           rs7338868
allele1                    A
allele2                    C
gkm_SVM            17.609279
kmer_SVM           12.627238
abs_gkm_SVM        17.609279
abs_kmer_SVM       12.627238
CADD               -0.132208
GWAVA_region            0.34
GWAVA_tss               0.38
GWAVA_unmatched         0.23
GERP                   -2.29
label                      1
match                   True
Name: 137, dtype: object
10


In [60]:
subset[5].values[0]

'T'

In [61]:
mapping[nucleotide]

'T'

# restart and test the bim

In [9]:
#let's check the excel file and make sure it matches
import pandas as pd
from tqdm import tqdm
import numpy as np
vars_list = pd.read_excel('/data1/lesliec/sarthak/data/joint_playground/dsQTL/41588_2015_BFng3331_MOESM26_ESM.xlsx', sheet_name='SuppTable1')
#let's load in the bim files
bims = {}
for i in range(1,23):
    bim_path = f'/data1/deyk/extras/1000G_BIMS_hg38/1000G.EUR.QC.{i}.bim'
    bim = pd.read_csv(bim_path, sep='\t', header=None)
    bims[i] = bim

genome = np.load('/data1/lesliec/sarthak/data/chrombpnet_test/hg38_tokenized.npz')

In [10]:
bim_rsids = {chrom: set(bims[chrom][1].values) for chrom in bims}

# Step 2: Check membership
def is_rsid_in_bim(row):
    chrom = int(row['chrom_hg19'][3:])
    rsid = row['SNPname2']
    return rsid in bim_rsids.get(chrom, set())

# Step 3: Apply the function to the dataframe
vars_list['match'] = vars_list.apply(is_rsid_in_bim, axis=1)

In [11]:
#let's redo it but fixed

not_matched = []
mapping = {7:'A', 8:'C', 9:'G', 10:'T'}
for i in tqdm(range(vars_list.shape[0])):
    if not vars_list['match'][i]:
        continue
    rsid = vars_list['SNPname2'][i]
    chrom = int(vars_list['chrom_hg19'][i][3:])
    subset = bims[chrom][bims[chrom][1].str.contains(rsid)]
    if subset.shape[0] == 0:
        continue
    nucleotide = genome[f'chr{chrom}'][subset[3].values[0]-1]
    if vars_list['allele2'][i] != mapping[nucleotide] and vars_list['allele1'][i] != mapping[nucleotide]:
        not_matched.append(i)
        if subset[4].values[0] != mapping[nucleotide] and subset[5].values[0] != mapping[nucleotide]:
            print(i)
            print(subset)
            print(vars_list.iloc[i])
            print(genome[f'chr{chrom}'][subset[3].values[0]-1])
            break

  1%|          | 226/28309 [00:33<1:09:57,  6.69it/s]


KeyboardInterrupt: 

In [16]:
bims[chrom].index

RangeIndex(start=0, stop=366088, step=1)

In [17]:
#let's make it faster
bim_pos = {
    chrom: dict(zip(bims[chrom][1].values, bims[chrom][3].index))
    for chrom in bims
}
bim_pos

{1: {'rs575272151': 0,
  'rs544419019': 1,
  'rs540538026': 2,
  'rs62635286': 3,
  'rs200579949': 4,
  'rs531730856': 5,
  'rs554008981': 6,
  'rs546169444': 7,
  'rs531646671': 8,
  'rs541940975': 9,
  'rs199856693': 10,
  'rs564003018': 11,
  'rs374029747': 12,
  'rs568149713': 13,
  'rs2691315': 14,
  'rs199745162': 15,
  'rs564023708': 16,
  'rs533090414': 17,
  'rs806731': 18,
  'rs542415070': 19,
  'rs559500163': 20,
  'rs528344458': 21,
  'rs551668143': 22,
  'rs565211799': 23,
  'rs559190862': 24,
  'rs575564077': 25,
  'rs2691277': 26,
  'rs530867301': 27,
  'rs141149254': 28,
  'rs552304420': 29,
  'rs569799965': 30,
  'rs3091274': 31,
  'rs114420996': 32,
  'rs547227933': 33,
  'rs62637817': 34,
  'rs541415601': 35,
  'rs527362206': 36,
  'rs184286948': 37,
  'rs62637820': 38,
  'rs557590765': 39,
  'rs62639100': 40,
  'rs80011619': 41,
  'rs552815263': 42,
  'rs181431124': 43,
  'rs62639104': 44,
  'rs181028663': 45,
  'rs545950684': 46,
  'rs571658168': 47,
  'rs532215594

In [18]:
matching_vars = vars_list[vars_list['match']].copy()
mapping = {7: 'A', 8: 'C', 9: 'G', 10: 'T'}
not_matched = []

for i, row in tqdm(matching_vars.iterrows(), total=matching_vars.shape[0]):
    chrom = int(row['chrom_hg19'][3:])
    rsid = row['SNPname2']
    
    # Position lookup
    bimrow = bim_pos.get(chrom, {}).get(rsid, None)
    if bimrow is None:
        continue  # shouldn't happen if match is True
    pos = bims[chrom].iloc[bimrow][3]  # 1-based position in the bim file
    
    # Get reference nucleotide
    ref_base = mapping[genome[f'chr{chrom}'][pos - 1]]

    # Compare to allele1 / allele2
    if row['allele1'] != ref_base and row['allele2'] != ref_base:
        altallele1 = bims[chrom].iloc[bimrow][4]
        altallele2 = bims[chrom].iloc[bimrow][5]
        if altallele1 != ref_base and altallele2 != ref_base:
            not_matched.append(i)
            print(i)
            print(subset)
            print(row)
            print(ref_base)
            break

  5%|▍         | 867/17515 [02:31<48:31,  5.72it/s]  


KeyboardInterrupt: 

In [23]:
#why is this slow? First part is instant
#load in genome to make it faster
genome_loaded = {}
for key in genome:
    genome_loaded[key] = genome[key][:]

In [ ]:
matching_vars = vars_list[vars_list['match']].copy()
mapping = {7: 'A', 8: 'C', 9: 'G', 10: 'T'}
not_matched = []

for i, row in tqdm(matching_vars.iterrows(), total=matching_vars.shape[0]):
    chrom = int(row['chrom_hg19'][3:])
    rsid = row['SNPname2']
    
    # Position lookup
    bimrow = bim_pos.get(chrom, {}).get(rsid, None)
    if bimrow is None:
        continue  # shouldn't happen if match is True
    pos = bims[chrom].iloc[bimrow][3]  # 1-based position in the bim file
    
    # Get reference nucleotide
    ref_base = mapping[genome_loaded[f'chr{chrom}'][pos - 1]]

    # Compare to allele1 / allele2
    if row['allele1'] != ref_base and row['allele2'] != ref_base:
        altallele1 = bims[chrom].iloc[bimrow][4]
        altallele2 = bims[chrom].iloc[bimrow][5]
        not_matched.append(i)
        if altallele1 != ref_base and altallele2 != ref_base:
            print(i)
            print(row)
            print(ref_base)
            print(altallele1, altallele2) #yeah so this is a problem?
            break

  8%|▊         | 1416/17515 [00:00<00:01, 10953.04it/s]

2188
chrom_hg18                   chr1
pos_hg18                120707881
SNPname1           chr1.120707881
chrom_hg19                   chr1
pos_hg19                120906358
SNPname2              rs182963377
allele1                         A
allele2                         G
gkm_SVM                   1.36771
kmer_SVM                 1.113394
abs_gkm_SVM               1.36771
abs_kmer_SVM             1.113394
CADD                     1.418225
GWAVA_region                 0.47
GWAVA_tss                    0.51
GWAVA_unmatched              0.91
GERP                         1.92
label                          -1
match                        True
Name: 2188, dtype: object
C
A G


In [28]:
#so in this example we have this rsid, but the bim says it's A and G but the website for this says T or C...

matching_vars = vars_list[vars_list['match']].copy()
mapping = {7: 'A', 8: 'C', 9: 'G', 10: 'T'}
not_matched = []
incorrect = []

for i, row in tqdm(matching_vars.iterrows(), total=matching_vars.shape[0]):
    chrom = int(row['chrom_hg19'][3:])
    rsid = row['SNPname2']
    
    # Position lookup
    bimrow = bim_pos.get(chrom, {}).get(rsid, None)
    if bimrow is None:
        continue  # shouldn't happen if match is True
    pos = bims[chrom].iloc[bimrow][3]  # 1-based position in the bim file
    
    # Get reference nucleotide
    ref_base = mapping[genome_loaded[f'chr{chrom}'][pos - 1]]

    # Compare to allele1 / allele2
    if row['allele1'] != ref_base and row['allele2'] != ref_base:
        altallele1 = bims[chrom].iloc[bimrow][4]
        altallele2 = bims[chrom].iloc[bimrow][5]
        not_matched.append(i)
        if altallele1 != ref_base and altallele2 != ref_base:
            # print(i)
            # print(row)
            # print(ref_base)
            # print(altallele1, altallele2) #yeah so this is a problem?
            incorrect.append(i)

100%|██████████| 17515/17515 [00:01<00:00, 11755.60it/s]


In [ ]:
len(incorrect) #8 is whatever, can exclude these!

8

In [ ]:
len(not_matched) #also such a small number so we can just exclude it!

9

In [31]:
print(incorrect)
print(not_matched)

[2188, 3202, 3273, 8619, 8620, 8621, 8623, 18331]
[137, 2188, 3202, 3273, 8619, 8620, 8621, 8623, 18331]


In [ ]:
#interesting that they tend to be the same other than the first example...
#easiest to just exclude these ones, then the bims and excel match with the reference genome!! 
for idx in not_matched:
    chrom = int(vars_list['chrom_hg19'][idx][3:])
    rsid = vars_list['SNPname2'][idx]
    bimrow = bim_pos.get(chrom, {}).get(rsid, None)
    pos = bims[chrom].iloc[bimrow][3]  # 1-based position in the bim file
    ref_base = mapping[genome_loaded[f'chr{chrom}'][pos - 1]]
    print(f'rsid: {rsid}, reference: {ref_base}, bimalleles: {bims[chrom].iloc[bimrow][4]}, {bims[chrom].iloc[bimrow][5]}, excelalleles: {vars_list["allele1"][idx]}, {vars_list["allele2"][idx]}')

#first one matches bim but not excel for some reason

rsid: rs7338868, reference: T, bimalleles: G, T, excelalleles: A, C
rsid: rs182963377, reference: C, bimalleles: A, G, excelalleles: A, G
rsid: rs9665316, reference: G, bimalleles: T, C, excelalleles: T, C
rsid: rs2790957, reference: T, bimalleles: A, G, excelalleles: A, G
rsid: rs2166258, reference: C, bimalleles: A, G, excelalleles: A, G
rsid: rs56074216, reference: G, bimalleles: T, C, excelalleles: T, C
rsid: rs7183752, reference: T, bimalleles: A, G, excelalleles: A, G
rsid: rs4778307, reference: T, bimalleles: A, C, excelalleles: A, C
rsid: rs469840, reference: C, bimalleles: G, A, excelalleles: G, A


In [38]:
vars_list

,chrom_hg18,pos_hg18,SNPname1,chrom_hg19,pos_hg19,SNPname2,allele1,allele2,gkm_SVM,kmer_SVM,abs_gkm_SVM,abs_kmer_SVM,CADD,GWAVA_region,GWAVA_tss,GWAVA_unmatched,GERP,label,match
0,chr1,846446,chr1.846446,chr1,856583,rs141671872,A,G,2.653531,7.938784,2.653531,7.938784,-1.522533,0.48,0.53,0.75,-0.682,1,False
1,chr1,901458,chr1.901458,chr1,911595,rs7417106,G,A,2.821422,6.027648,2.821422,6.027648,1.739152,0.29,0.36,0.89,2.150,1,True
2,chr1,1176365,chr1.1176365,chr1,1186502,rs6603785,T,A,-7.167236,-10.870018,7.167236,10.870018,-0.091378,0.42,0.49,0.66,0.451,1,True
3,chr1,1217275,chr1.1217275,chr1,1227412,rs112571439,A,G,-3.957382,2.387491,3.957382,2.387491,1.127472,0.34,0.43,0.49,2.430,1,False
4,chr1,1580438,chr1.1580438,chr1,1590575,rs9661285,A,G,-4.785596,-6.827102,4.785596,6.827102,0.665464,0.38,0.35,0.86,1.640,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28304,chr9,140155407,chr9.140155407,chr9,141035586,rs7852209,T,C,0.444036,-1.583811,0.444036,1.583811,-0.606667,0.19,0.22,0.25,-0.890,-1,True
28305,chr9,140181121,chr9.140181121,chr9,141061300,rs4066694,T,C,1.363332,1.703144,1.363332,1.703144,0.489617,0.42,0.14,0.16,0.118,-1,True
28306,chr9,140213801,chr9.140213801,chr9,141093980,rs7046723,A,G,-3.102628,3.391616,3.102628,3.391616,-0.744948,0.23,0.18,0.35,0.000,-1,False
28307,chr9,140219484,chr9.140219484,chr9,141099663,rs12340415,G,C,-4.628759,-2.087131,4.628759,2.087131,-0.809512,0.41,0.27,0.15,0.000,-1,True


In [44]:
#so let's exclude these 9 rsids and save the csv

final_vars_list = vars_list.drop(index=not_matched)
final_vars_list = final_vars_list[final_vars_list['match']].reset_index(drop=True)
final_vars_list

,chrom_hg18,pos_hg18,SNPname1,chrom_hg19,pos_hg19,SNPname2,allele1,allele2,gkm_SVM,kmer_SVM,abs_gkm_SVM,abs_kmer_SVM,CADD,GWAVA_region,GWAVA_tss,GWAVA_unmatched,GERP,label,match
0,chr1,901458,chr1.901458,chr1,911595,rs7417106,G,A,2.821422,6.027648,2.821422,6.027648,1.739152,0.29,0.36,0.89,2.150,1,True
1,chr1,1176365,chr1.1176365,chr1,1186502,rs6603785,T,A,-7.167236,-10.870018,7.167236,10.870018,-0.091378,0.42,0.49,0.66,0.451,1,True
2,chr1,3359707,chr1.3359707,chr1,3369847,rs55665824,G,A,-5.249735,-4.336111,5.249735,4.336111,0.932819,0.43,0.47,0.65,0.354,1,True
3,chr1,11731989,chr1.11731989,chr1,11809402,rs11121820,G,T,-1.585787,-1.229891,1.585787,1.229891,0.197824,0.37,0.32,0.40,-4.200,1,True
4,chr1,11964848,chr1.11964848,chr1,12042261,rs2236055,A,G,2.366937,1.784808,2.366937,1.784808,3.482362,0.23,0.22,0.64,2.110,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17501,chr9,140058004,chr9.140058004,chr9,140938183,rs3750506,A,G,0.487228,1.529274,0.487228,1.529274,-0.580919,0.22,0.33,0.38,-7.050,-1,True
17502,chr9,140073118,chr9.140073118,chr9,140953297,rs10780199,G,A,2.804388,5.312007,2.804388,5.312007,0.245404,0.31,0.40,0.29,0.961,-1,True
17503,chr9,140155407,chr9.140155407,chr9,141035586,rs7852209,T,C,0.444036,-1.583811,0.444036,1.583811,-0.606667,0.19,0.22,0.25,-0.890,-1,True
17504,chr9,140181121,chr9.140181121,chr9,141061300,rs4066694,T,C,1.363332,1.703144,1.363332,1.703144,0.489617,0.42,0.14,0.16,0.118,-1,True


In [45]:
#find how many in label column are 1
final_vars_list['label'].value_counts()

label
-1    17071
 1      435
Name: count, dtype: int64

In [46]:
final_vars_list['match'].value_counts()

match
True    17506
Name: count, dtype: int64

In [47]:
#and we can now save it out
final_vars_list.to_csv('/data1/lesliec/sarthak/data/joint_playground/dsQTL/filtered_dsqtls.txt', index=False, sep='\t')

In [48]:
#print hostname
!hostname

iscb019.mskcc.org


# could the issue be chromosomes??

In [ ]:
import pandas as pd
bims = {}
for i in range(1,23):
    bim_path = f'/data1/deyk/extras/1000G_BIMS_hg38/1000G.EUR.QC.{i}.bim'
    bim = pd.read_csv(bim_path, sep='\t', header=None)
    bims.update(dict(zip(bim[1], bim[0])))
    # for j in range(bim.shape[0]):
    #     bims[bim.iloc[j][1]] = bim.iloc[j][0]
    print('done with chrom', i)
len(bims) #almost 1 million!!

done with chrom 1
done with chrom 2
done with chrom 3
done with chrom 4
done with chrom 5
done with chrom 6
done with chrom 7
done with chrom 8
done with chrom 9
done with chrom 10
done with chrom 11
done with chrom 12
done with chrom 13
done with chrom 14
done with chrom 15
done with chrom 16
done with chrom 17
done with chrom 18
done with chrom 19
done with chrom 20
done with chrom 21
done with chrom 22


9991229

In [6]:
bims

{'rs575272151': 1,
 'rs544419019': 1,
 'rs540538026': 1,
 'rs62635286': 1,
 'rs200579949': 1,
 'rs531730856': 1,
 'rs554008981': 1,
 'rs546169444': 1,
 'rs531646671': 1,
 'rs541940975': 1,
 'rs199856693': 1,
 'rs564003018': 1,
 'rs374029747': 1,
 'rs568149713': 1,
 'rs2691315': 1,
 'rs199745162': 1,
 'rs564023708': 1,
 'rs533090414': 1,
 'rs806731': 1,
 'rs542415070': 1,
 'rs559500163': 1,
 'rs528344458': 1,
 'rs551668143': 1,
 'rs565211799': 1,
 'rs559190862': 1,
 'rs575564077': 1,
 'rs2691277': 1,
 'rs530867301': 1,
 'rs141149254': 1,
 'rs552304420': 1,
 'rs569799965': 1,
 'rs3091274': 1,
 'rs114420996': 1,
 'rs547227933': 1,
 'rs62637817': 1,
 'rs541415601': 1,
 'rs527362206': 1,
 'rs184286948': 1,
 'rs62637820': 1,
 'rs557590765': 1,
 'rs62639100': 1,
 'rs80011619': 1,
 'rs552815263': 1,
 'rs181431124': 1,
 'rs62639104': 1,
 'rs181028663': 1,
 'rs545950684': 1,
 'rs571658168': 1,
 'rs532215594': 1,
 'rs558384541': 1,
 'rs13328700': 1,
 'rs13328684': 1,
 'rs540748103': 1,
 'rs550898

In [8]:
#now read the excel file
vars_list = pd.read_excel('/data1/lesliec/sarthak/data/joint_playground/dsQTL/41588_2015_BFng3331_MOESM26_ESM.xlsx', sheet_name='SuppTable1')
#now loop through, see how many are there, then see if chromosome matches
unmatched = []
for i in range(vars_list.shape[0]):
    rsid = vars_list['SNPname2'][i]
    chrom = vars_list['chrom_hg19'][i]
    if rsid not in bims:
        unmatched.append(i)
        continue
    if bims[rsid] != chrom[3:]:
        print('chromosome mismatch', rsid, chrom, bims[rsid])
        break

chromosome mismatch rs7417106 chr1 1


In [11]:
unmatched = []
chr_mismatch = []
for i in range(vars_list.shape[0]):
    rsid = vars_list['SNPname2'][i]
    chrom = vars_list['chrom_hg19'][i]
    if rsid not in bims:
        unmatched.append(i)
        continue
    if bims[rsid] != int(chrom[3:]):
        print('chromosome mismatch', rsid, chrom, bims[rsid])
        # break
        chr_mismatch.append(i)
print(len(unmatched), len(chr_mismatch))

chromosome mismatch rs58014911 chr1 18
10793 1


In [ ]:
#so we probably threw away the one mismatch, that's fine, good to see on same chromosome tho! Can say it's not an issue!